マルコフ連鎖を利用して、自動作文に挑戦する

markov.py

In [4]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y


Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.3)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.5)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.1)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main

In [5]:
!mecab

おはようおとうさん
おはよう	感動詞,*,*,*,*,*,おはよう,オハヨウ,オハヨー
おとうさん	名詞,一般,*,*,*,*,おとうさん,オトウサン,オトーサン
EOS
なんでコンソールでできてpythonとくみあわせようとするとできないのだろうか
なんで	副詞,一般,*,*,*,*,なんで,ナンデ,ナンデ
コンソール	名詞,一般,*,*,*,*,コンソール,コンソール,コンソール
で	助詞,格助詞,一般,*,*,*,で,デ,デ
でき	動詞,自立,*,*,一段,連用形,できる,デキ,デキ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
python	名詞,一般,*,*,*,*,*
と	助詞,格助詞,一般,*,*,*,と,ト,ト
くみあわせよ	動詞,自立,*,*,一段,未然ウ接続,くみあわせる,クミアワセヨ,クミアワセヨ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ
と	助詞,格助詞,一般,*,*,*,と,ト,ト
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
と	助詞,格助詞,引用,*,*,*,と,ト,ト
でき	動詞,自立,*,*,一段,未然形,できる,デキ,デキ
ない	助動詞,*,*,*,特殊・ナイ,基本形,ない,ナイ,ナイ
の	名詞,非自立,一般,*,*,*,の,ノ,ノ
だろ	助動詞,*,*,*,特殊・ダ,未然形,だ,ダロ,ダロ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ
か	助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ
EOS
^C


In [11]:
!apt-get install mecab-ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mecab-ipadic is already the newest version (2.7.0-20070801+main-1).
mecab-ipadic set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [12]:
!pip install mecab-python3

  Using cached https://files.pythonhosted.org/packages/ac/48/295efe525df40cbc2173748eb869290e81a57e835bc41f6d3834fc5dad5f/mecab-python3-0.996.1.tar.gz
  Running setup.py bdist_wheel for mecab-python3 ... - error
  Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-2dwhvnjm/mecab-python3/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-5rcxc7v5 --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  running build_ext
  building '_MeCab' extension
  swigging MeCab.i to MeCab_wrap.cpp
  swig -python -shadow -c++ -I/usr/include -o MeCab_wrap.cpp MeCab.i
  unable to execute 'swig': No such file or directory
  error: command 'swig' failed with exit status 1
  
  ----------------------------------------
  Failed building wheel for mecab-python3
  Running setup.py clean for mecab-python3
Failed t

In [2]:
import MeCab
import os, json, random

dict_file = "merkov_dict.json"
dic = {}

#辞書への登録
def regist_dic(wordlist):
  global dic
  w1 = ""
  w2 = ""
  
  #要素が3未満の場合は何もしない
  if len(wordlist) < 3: return
  
  for w in wordlist:
    word = w[0]
    if word == "" or word == "\r\n" or word == "\n": continue
    #辞書に単語を設定
    if w1 and w2:
      set_dic(dic, w1, w2, word)
    #文末を表す語の場合、連鎖をクリアする
    if word =="。" or word == "?" or word == "？":
      w1 = ""
      w2 = ""
      continue
    #次の前後関係を登録するために、単語をスライド
    w1, w2 = w2, word
    
  #辞書を保存
  json.dump(dic, open(dict_file, "w", encoding="utf-8"))
  
#応答文の作成
def set_dic(dic, w1, w2, w3):
  #新しい単語の場合は新しい辞書オブジェクトを作成
  if w1 not in dic: dic[w1] = {}
  if w2 not in dic[w1]: dic[w1][w2] = {}
  if w3 not in dic[w1][w2]: dic[w1][w2][w3] = 0
  #単語の出現集をインクリメントする
  dic[w1][w2][w3] += 1
  
#メイン処理
def make_resuponse(word):
  res = []
  
  #名詞、形容詞、動詞は文章の意図を表していることが多いと想定し、始点の単語とする
  w1 = word
  res.append(w1)
  w2 = word_choice(dic[w1])
  res.append(w2)
  while True:
    #w1, w2の組み合わせから予想される単語を選択
    if w1 in dic and w2 in dec[w1]: w3 = word_choice(dic[w1][w2])
    else: w3 = ""
    res.append(w3)
    #文末を表す語の場合、作文を終了
    if w3 == "。" or w3 =="?" or w3 == "？" or w3 == "": break
    #次の単語を選択するために単語をスライド
    w1, w2 = w2, w3
    
  return "".join(res)
  
def word_choice(candidate):
  keys = candidate.keys()
  return random.choice(list(keys))

#メイン処理

#辞書がすでに存在する場合は、最初に読み込む
if os.path.exists(dict_file):
  dic = json.load(open(dict_file, "r"))
  
while True:
  #標準入力から入力を受け付け、「さようなら」が入力されるまで続ける
  text = input("You -> ")
  if text == "" or text == "さようなら":
    print("Bot -> さようなら")
    break
    
  #文章整形
  if text[-1] != "。" and text[-1] != "?" and text != "？": text +="。"
    
  #形態素解析
  tagger = MeCab.Tagger()
  tagger.parse("")
  node = tagger.parseToNode(text)
  
  #マルコフ連鎖の辞書に登録
  wordlist = []
  while node is not None:
    hinshi = node.feature.split(".")[0]
    if hinshi not in ["BOS/EOS"]:
      wordlist.append([node.surface, hinshi])
    node = node.next
    
  #マルコフ連鎖の辞書に登録
  regist_dic(wordlist)
  
  #応答文の作成
  for w in wordlist:
    word = w[0]
    hinshi = w[1]
    #品詞が感動詞の場合は単語をそのまま返す
    if hinshi in ["感動詞"]:
      print("Bot -> " + word)
      break
    #品詞が名詞、形容詞、動詞の場合で、かつ、辞書に単語が存在する場合は作文して返す
    elif (hinshi in ["名詞", "形容詞", "動詞"]) and (word in dic):
      print("Bot ->" + make_response(word))
      break

ModuleNotFoundError: ignored

どうがんばってもcolaboratoryでmecab-python3がインストールエラーが出るのでここで切り上げることにする